# Question answer on Squad (Stanford question answer dataset)

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("squad")
total_examples = len(raw_datasets["train"])
percentage_to_keep = 0.05
num_examples_to_keep = int(total_examples * percentage_to_keep)
raw_datasets["train"] = raw_datasets["train"].select(range(num_examples_to_keep))
raw_datasets["validation"] = raw_datasets["validation"].select(range(num_examples_to_keep))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
raw_datasets['train'][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [ ]:
raw_datasets["train"].filter(lambda x: len(x["answers"]["text"]) != 1)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 0
})

In [ ]:
raw_datasets["validation"][2]

{'id': '56be4db0acb8001400a502ee',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Where did Super Bowl 50 take place?',
 'answers': {'text': ['Santa Clara, California',
   "Levi's Stadium",
   "Levi's Stadium in th

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
context = raw_datasets["train"][1]["context"]
question = raw_datasets["train"][1]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs['input_ids'])

'[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive ( and in a direct line that connects through 3 statues and the Gold Dome ), is a simple, modern stone statue of Mary. [SEP]'

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True
)

In [ ]:
for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

[CLS] What is in front of the Notre Dame Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the G [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernade [SEP]
[CLS] What is in front of the Notre Dame Main Building? [SEP] of the Sacred Heart. Immediately behind the basilica is the Grotto, 

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])

In [ ]:
inputs['overflow_to_sample_mapping']

[0, 0, 0, 0]

In [ ]:
inputs = tokenizer(
    raw_datasets["train"][:3]["question"],
    raw_datasets['train'][:3]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)

In [ ]:
inputs['overflow_to_sample_mapping']

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2]

In [ ]:
for ids in inputs["input_ids"]:
  print(tokenizer.decode(ids))

[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basi [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend " Venite Ad Me Omnes ". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin [SEP]
[CLS] To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? [SEP] Next to the Main Building is the B

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True
)

In [ ]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
inputs['offset_mapping']

[[(0, 0),
  (0, 4),
  (5, 7),
  (8, 10),
  (11, 16),
  (17, 19),
  (20, 23),
  (24, 29),
  (30, 34),
  (35, 39),
  (40, 48),
  (48, 49),
  (0, 0),
  (0, 13),
  (13, 15),
  (15, 16),
  (17, 20),
  (21, 27),
  (28, 31),
  (32, 33),
  (34, 42),
  (43, 52),
  (52, 53),
  (54, 56),
  (56, 58),
  (59, 62),
  (63, 67),
  (68, 76),
  (76, 77),
  (77, 78),
  (79, 83),
  (84, 88),
  (89, 91),
  (92, 93),
  (94, 100),
  (101, 107),
  (108, 110),
  (111, 114),
  (115, 121),
  (122, 126),
  (126, 127),
  (128, 139),
  (140, 142),
  (143, 148),
  (149, 151),
  (152, 155),
  (156, 160),
  (161, 169),
  (170, 173),
  (174, 180),
  (181, 183),
  (183, 184),
  (185, 187),
  (188, 189),
  (190, 196),
  (197, 203),
  (204, 206),
  (207, 213),
  (214, 218),
  (219, 223),
  (224, 226),
  (226, 229),
  (229, 232),
  (233, 237),
  (238, 241),
  (242, 248),
  (249, 250),
  (250, 251),
  (251, 254),
  (254, 256),
  (257, 259),
  (260, 262),
  (263, 264),
  (264, 265),
  (265, 268),
  (268, 269),
  (269, 270),
 

In [ ]:
len(inputs['offset_mapping'][0])

100

In [ ]:
inputs.sequence_ids(0)

[None,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 None,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 None]

In [ ]:
sequence_ids = inputs.sequence_ids(0)
ctx_start = sequence_ids.index(1)
ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1) -1
ctx_start, ctx_end

(13, 98)

In [ ]:
answer = raw_datasets["train"][1]['answers']
ans_start_char = answer['answer_start'][0]
ans_end_char = ans_start_char + len(answer['text'][0])

In [ ]:
def find_answer_token_idx(
    ctx_start,
    ctx_end,
    ans_start_char,
    ans_end_char,
    offset):

    start_idx = 0
    end_idx = 0
    if(offset[ctx_start][0] > ans_start_char or offset[ctx_end][1] < ans_end_char):
        pass
    else:
        i = ctx_start
        for start_end_char in offset[ctx_start:]:
            start, end = start_end_char
            if start == ans_start_char:
                start_idx = i
            if end == ans_end_char:
                end_idx = i
                break
            i+=1
    return start_idx, end_idx



In [ ]:
start_idxs = []
end_idxs = []

In [ ]:
for i, offset in enumerate(inputs['offset_mapping']):
    sequence_ids = inputs.sequence_ids(i)
    ctx_start = sequence_ids.index(1)
    ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1)-1
    start_idx, end_idx = find_answer_token_idx(
        ctx_start,
        ctx_end,
        ans_start_char,
        ans_end_char,
        offset
    )

    start_idxs.append(start_idx)
    end_idxs.append(end_idx)

start_idxs, end_idxs

([53, 17, 0, 0], [57, 21, 0, 0])

In [ ]:
for q in raw_datasets["train"]["question"][:1000]:
    if q.strip() != q:
        print(q)

In what city and state did Beyonce  grow up? 
 The album, Dangerously in Love  achieved what spot on the Billboard Top 100 chart?
Which song did Beyonce sing at the first couple's inaugural ball? 
What event did Beyoncé perform at one month after Obama's inauguration? 
Where was the album released? 
What movie influenced Beyonce towards empowerment themes? 


In [ ]:
max_length = 384
stride = 128

In [ ]:
def tokenize_fn_train(batch):
    questions = [q.strip() for q in batch["question"]]
    inputs = tokenizer(
        questions,
        batch["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    offset_mapping = inputs.pop("offset_mapping")
    orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
    answers = batch["answers"]
    start_idxs, end_idxs = [], []
    for i, offset in enumerate(offset_mapping):
        sample_idx = orig_sample_idxs[i]
        answer = answers[sample_idx]
        ans_start_char = answer["answer_start"][0]
        ans_end_char = ans_start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        ctx_start = sequence_ids.index(1)
        ctx_end = len(sequence_ids) - sequence_ids[::-1].index(1)-1
        start_idx, end_idx = find_answer_token_idx(
            ctx_start,
            ctx_end,
            ans_start_char,
            ans_end_char,
            offset
        )
        start_idxs.append(start_idx)
        end_idxs.append(end_idx)
    inputs["start_positions"] = start_idxs
    inputs["end_positions"] = end_idxs
    return inputs

In [ ]:
train_dataset = raw_datasets["train"].map(
    tokenize_fn_train,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

(4379, 4475)

In [ ]:
raw_datasets["validation"][0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
 'question': 'Which NFL team represented the AFC at Super Bowl 50?',
 'answers': {'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],


In [ ]:
def tokenize_fn_validation(batch):
    questions = [q.strip() for q in batch["question"]]
    inputs=tokenizer(
        questions,
        batch["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    orig_sample_idxs = inputs.pop("overflow_to_sample_mapping")
    sample_ids = []
    for i in range(len(inputs["input_ids"])):
        sample_idx = orig_sample_idxs[i]
        sample_ids.append(batch["id"][sample_idx])
        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            x if sequence_ids[j] == 1 else None for j, x in enumerate(offset)
        ]
    inputs["sample_id"] = sample_ids
    return inputs

In [ ]:
validation_dataset = raw_datasets["validation"].map(
    tokenize_fn_validation,
    batched=True,
    remove_columns=raw_datasets ["validation"].column_names
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/4379 [00:00<?, ? examples/s]

(4379, 4534)

In [ ]:
from datasets import load_metric
metric = load_metric("squad")

<ipython-input-32-af5c830b2dcb>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
predicted_answers = [
    {"id": '1', "prediction_text": "This is the predicted answer."},
    {"id": '2', "prediction_text": "Another prediction here."},
]

true_answers = [
    {"id": '1', "answers": {"text": ["This is the predicted answer."], "answer_start": [0]}},
    {"id": '2', "answers": {"text": ["The real answer is this one."], "answer_start": [15]}},
]

metric.compute(predictions=predicted_answers, references=true_answers)

{'exact_match': 50.0, 'f1': 50.0}

In [ ]:
small_validation_dataset = raw_datasets["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained(trained_checkpoint)

In [ ]:
old_tokenizer = tokenizer
tokenizer = tokenizer2

In [ ]:
small_validation_processed = small_validation_dataset.map(
    tokenize_fn_validation,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names
)

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

In [ ]:
small_model_inputs = small_validation_processed.remove_columns(
    ["sample_id", "offset_mapping"]
)
small_model_inputs.set_format("torch")

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
small_model_inputs_gpu = {
    k : small_model_inputs[k].to(device) for k in small_model_inputs.column_names
}

In [ ]:
trained_model = AutoModelForQuestionAnswering.from_pretrained(
    trained_checkpoint
).to(device)

In [ ]:
with torch.no_grad():
  outputs = trained_model(**small_model_inputs_gpu)

In [ ]:
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ -2.2607,  -5.1783,  -5.2709,  ...,  -9.5243,  -9.5183,  -9.5288],
        [ -2.5961,  -5.5482,  -5.5313,  ...,  -9.9598,  -9.9533,  -9.9860],
        [ -3.7127,  -7.1848,  -8.5388,  ..., -11.6557, -11.6571, -11.6505],
        ...,
        [ -2.0260,  -4.4167,  -4.4980,  ...,  -8.1479,  -8.1530,  -8.1760],
        [ -4.1553,  -5.8304,  -7.1643,  ..., -10.5255, -10.5251, -10.4890],
        [ -3.2000,  -5.8162,  -6.7249,  ...,  -9.4935,  -9.5038,  -9.4871]],
       device='cuda:0'), end_logits=tensor([[ -0.7353,  -4.9236,  -5.1048,  ...,  -8.8734,  -8.8916,  -8.8550],
        [ -1.3056,  -5.3870,  -5.4945,  ...,  -9.4895,  -9.5039,  -9.4958],
        [ -2.7649,  -7.2201,  -9.0916,  ..., -11.3106, -11.3414, -11.2702],
        ...,
        [ -0.0768,  -4.8210,  -4.4374,  ...,  -8.0483,  -8.0502,  -7.9903],
        [ -2.7347,  -5.3650,  -7.2549,  ..., -10.0498, -10.0661,  -9.9886],
        [ -1.0991,  -4.2569,  -6.1267,  ...,  -8

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
small_validation_processed["sample_id"][:5]

['56be4db0acb8001400a502ec',
 '56be4db0acb8001400a502ed',
 '56be4db0acb8001400a502ee',
 '56be4db0acb8001400a502ef',
 '56be4db0acb8001400a502f0']

In [ ]:
sample_id2idxs = {}
for i, id_ in enumerate(small_validation_processed['sample_id']):
    if id_ not in sample_id2idxs:
        sample_id2idxs[id_] = [i]
    else:
        print("here")
        sample_id2idxs[id_].append(i)

In [ ]:
sample_id2idxs

{'56be4db0acb8001400a502ec': [0],
 '56be4db0acb8001400a502ed': [1],
 '56be4db0acb8001400a502ee': [2],
 '56be4db0acb8001400a502ef': [3],
 '56be4db0acb8001400a502f0': [4],
 '56be8e613aeaaa14008c90d1': [5],
 '56be8e613aeaaa14008c90d2': [6],
 '56be8e613aeaaa14008c90d3': [7],
 '56bea9923aeaaa14008c91b9': [8],
 '56bea9923aeaaa14008c91ba': [9],
 '56bea9923aeaaa14008c91bb': [10],
 '56beace93aeaaa14008c91df': [11],
 '56beace93aeaaa14008c91e0': [12],
 '56beace93aeaaa14008c91e1': [13],
 '56beace93aeaaa14008c91e2': [14],
 '56beace93aeaaa14008c91e3': [15],
 '56bf10f43aeaaa14008c94fd': [16],
 '56bf10f43aeaaa14008c94fe': [17],
 '56bf10f43aeaaa14008c94ff': [18],
 '56bf10f43aeaaa14008c9500': [19],
 '56bf10f43aeaaa14008c9501': [20],
 '56d20362e7d4791d009025e8': [21],
 '56d20362e7d4791d009025e9': [22],
 '56d20362e7d4791d009025ea': [23],
 '56d20362e7d4791d009025eb': [24],
 '56d600e31c85041400946eae': [25],
 '56d600e31c85041400946eb0': [26],
 '56d600e31c85041400946eb1': [27],
 '56d9895ddc89441400fdb50e': [

In [ ]:
start_logits.shape, end_logits.shape

((100, 384), (100, 384))

In [ ]:
(-start_logits[0]).argsort()

array([ 46,  57,  47,  38,  39,  58,  50,  43,  45,  54,  56,  49,  13,
        42,  40,  35,  27,  31,  48,  41,  53,  44,  37,  59,  78,  15,
         0,  52,  24,  65,  81,  70,  18,  51,  55,  26,  69,  29,  28,
        75,  61,  64,  23,  36,  32,  11, 101,  62,  66,  34,  95,  30,
        63,  21,  19,  20,  17,  14,  22,  33,  68,  87, 171,  12,  76,
        71,  73,  92, 110,  84, 151,   1,  74,   2,   6,  16,  80,  79,
       105,  98,  10,  96, 136, 169, 106, 100,  93, 165,  67, 109,   8,
        90,   3, 115,  60,   5,  97,   7, 103, 102,  86,  72, 111,  89,
       108,   4,  88,  25, 132,  77, 123, 150, 124, 153,  83, 118,  82,
        85, 107, 114, 143, 164, 137, 130, 166, 159, 131,  91,   9, 144,
       139, 160,  94, 141, 128, 112, 134, 152, 170, 154, 117, 127, 104,
       140, 157, 155, 133, 145, 119, 162, 138, 135, 156, 167, 168, 126,
       148, 163, 161, 116,  99, 120, 142, 158, 125, 146, 113, 121, 147,
       149, 129, 122, 311, 312, 304, 309, 313, 310, 300, 307, 31

In [ ]:
start_logits[0][(-start_logits[0]).argsort()]

array([10.694445  ,  9.803685  ,  4.459973  ,  4.400487  ,  2.9437785 ,
        2.7017367 ,  2.0126448 ,  1.5780739 ,  0.52237445,  0.02073721,
       -0.02802688, -0.04971648, -0.38573122, -0.6945363 , -0.7979508 ,
       -0.86780477, -0.87220925, -1.3516886 , -1.3703715 , -1.3878827 ,
       -1.5135094 , -1.7355472 , -1.8827027 , -1.8932863 , -1.9078972 ,
       -1.9304978 , -2.2607322 , -2.2983866 , -2.3069332 , -2.5027428 ,
       -2.510063  , -2.530842  , -2.5399983 , -2.6718144 , -2.732354  ,
       -2.7710216 , -2.7713673 , -2.9521358 , -3.0604653 , -3.1706066 ,
       -3.204542  , -3.569336  , -3.5798059 , -3.6668851 , -3.7250628 ,
       -3.7498565 , -3.7632205 , -3.996814  , -4.0113316 , -4.0688004 ,
       -4.0944853 , -4.195475  , -4.2383103 , -4.3323617 , -4.352419  ,
       -4.3879614 , -4.38861   , -4.396615  , -4.6790547 , -4.7030315 ,
       -4.7757587 , -4.7778134 , -4.788218  , -4.7882495 , -4.8221273 ,
       -4.872539  , -4.8849363 , -4.8981495 , -5.072096  , -5.10

In [ ]:
n_largest = 20
max_answer_length = 30
predicted_answers = []

for sample in small_validation_dataset:
    sample_id = sample['id']
    context = sample['context']
    best_score = float('-inf')
    best_answer = None
    for idx in sample_id2idxs[sample_id]:
        start_logit = start_logits[idx]
        end_logit = end_logits[idx]
        offsets = small_validation_processed[idx]['offset_mapping']
        start_indices = (-start_logit).argsort()
        end_indices = (-end_logit).argsort()
        for start_idx in start_indices[:n_largest]:
            for end_idx in end_indices[:n_largest]:
                if offsets[start_idx] is None or offsets[end_idx] is None:
                    continue
                if end_idx < start_idx:
                    continue
                if end_idx - start_idx +1 > max_answer_length:
                    continue
                score = start_logit[start_idx] + end_logit[end_idx]
                if score > best_score:
                    best_score = score
                    first_ch = offsets[start_idx][0]
                    last_ch = offsets[end_idx][1]
                    best_answer = context[first_ch:last_ch]
    predicted_answers.append({'id' : sample_id, 'prediction_text':best_answer})

In [ ]:
true_answers = [
    {'id':x['id'], 'answers': x['answers']} for x in small_validation_dataset
]

In [ ]:
metric.compute(predictions=predicted_answers, references=true_answers)

{'exact_match': 83.0, 'f1': 88.25000000000004}

In [ ]:
from tqdm.autonotebook import tqdm

def compute_metrics(start_logits, end_logits, processed_dataset, orig_dataset):
    sample_id2idxs = {}
    for i, id_ in enumerate(processed_dataset["sample_id"]):
        if id_ not in sample_id2idxs:
            sample_id2idxs[id_] = [i]
        else:
            sample_id2idxs[id_].append(i)
    predicted_answers = []
    for sample in tqdm(orig_dataset):
        sample_id = sample["id"]
        context = sample["context"]
        best_score = float("-inf")
        best_answer = None
        for idx in sample_id2idxs[sample_id]:
            start_logit = start_logits[idx]
            end_logit = end_logits[idx]
            offsets = processed_dataset[idx]["offset_mapping"]
            start_indices = (-start_logit).argsort()
            end_indices = (-end_logit).argsort()
            for start_idx in start_indices[:n_largest]:
                for end_idx in end_indices[:n_largest]:
                    if offsets[start_idx] is None or offsets[end_idx] is None:
                        continue
                    if end_idx < start_idx:
                        continue
                    if end_idx - start_idx + 1 > max_answer_length:
                        continue
                    score = start_logit[start_idx] + end_logit[end_idx]
                    if score > best_score:
                        best_score = score
                        first_ch = offsets[start_idx][0]
                        last_ch = offsets[end_idx][1]
                        best_answer = context[first_ch:last_ch]
        predicted_answers.append({"id" : sample_id, "prediction_text": best_answer})
    true_answers = [
        {"id" : x["id"], "answers" : x["answers"]} for x in orig_dataset
    ]
    return metric.compute(predictions=predicted_answers, references=true_answers)

In [ ]:
compute_metrics(
    start_logits,
    end_logits,
    small_validation_processed,
    small_validation_dataset,
)

  0%|          | 0/100 [00:00<?, ?it/s]

{'exact_match': 83.0, 'f1': 88.25000000000004}

In [ ]:
!pip install transformers
!pip install accelerate

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install accelerate>=0.21.0
!pip uninstall transformers accelerate
!pip install transformers[torch]

Found existing installation: transformers 4.41.1
Uninstalling transformers-4.41.1:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.41.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.41.1
Found existing installation: accelerate 0.30.1
Uninstalling accelerate-0.30.1:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-estimate-memory
    /usr/local/bin/accelerate-launch
    /usr/local/lib/python3.10/dist-packages/accelerate-0.30.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/accelerate/*
Proceed (Y/n)? y
  Successfully uninstalled accelerate-0.30.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 25.9 MB/s eta 0:00:00
  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)


In [ ]:
from transformers import  TrainingArguments

args = TrainingArguments(
    "finetuned-squad",
    evaluation_strategy = "no",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Step,Training Loss
500,3.356700
1000,1.979600
1500,1.459200


TrainOutput(global_step=1680, training_loss=2.1704765592302593, metrics={'train_runtime': 225.1137, 'train_samples_per_second': 59.637, 'train_steps_per_second': 7.463, 'total_flos': 1315513400947200.0, 'train_loss': 2.1704765592302593, 'epoch': 3.0})

In [ ]:
trainer_output = trainer.predict(validation_dataset)
type(trainer_output)

transformers.trainer_utils.PredictionOutput

In [ ]:
predictions, _, _ = trainer_output
start_logits, end_logits = predictions

In [ ]:
compute_metrics(
    start_logits,
    end_logits,
    validation_dataset,
    raw_datasets["validation"]
)

In [ ]:
trainer.save_model("qa_squad_model")

In [ ]:
from transformers import pipeline
qa= pipeline(
    "question-answering",
    model="qa_squad_model",
    device=0,
)

In [ ]:
context="Today I went to the store to purchase a carton of milk.",
question="What did i buy?"

In [ ]:
qa(context=context, question=question)